# Capítulo 5 — Enriquecimento com UN-Habitat Urban Data Portal

Este notebook implementa as etapas de enriquecimento de dados descritas para o projeto **"Cities of Tomorrow"**:

1. Acessar os dados do portal oficial da **UN-Habitat Urban Indicators Database**.
2. Carregar os arquivos baixados (CSV/Excel) para os indicadores:
   - Urban slums
   - Access to water
   - Access to sanitation
   - Green area per capita
   - PM2.5 (urban)
   - Urban population growth
   - Land consumption rate
3. Padronizar país, ano e valores numéricos.
4. Fazer **merge** com a base existente `wdi_merged.csv`.
5. Salvar um arquivo enriquecido `wdi_merged_unhabitat.csv` para uso nos capítulos seguintes.

> ⚠️ **Importante:**  
> Este notebook assume que você já fez o download manual dos arquivos no portal da UN‑Habitat e os salvou com os nomes indicados nas variáveis de caminho.


In [ ]:
# ============================================================
# CAPÍTULO 5 — ENRIQUECIMENTO COM UN-HABITAT URBAN DATA PORTAL
# ============================================================

import pandas as pd
import numpy as np

# ---------------------------------------------
# 1) Configuração de caminhos dos arquivos
#    -> ajuste os caminhos conforme o seu Lakehouse/Fabric
# ---------------------------------------------

# Dataset principal (Kaggle + WDI já integrado nos capítulos anteriores)
# Se já tiver um df_merged vindo de células anteriores, você pode pular este load.
PATH_MAIN = "/mnt/data/wdi_merged.csv"

# Arquivos baixados manualmente do UN-Habitat (ex.: Excel/CSV)
# Dica: salve com nomes simples no Lakehouse/Files e ajuste aqui.
PATH_SLUMS        = "/mnt/data/unhabitat_slums.xlsx"
PATH_WATER        = "/mnt/data/unhabitat_water_access.xlsx"
PATH_SANITATION   = "/mnt/data/unhabitat_sanitation_access.xlsx"
PATH_GREEN        = "/mnt/data/unhabitat_green_area_per_capita.xlsx"
PATH_PM25         = "/mnt/data/unhabitat_pm25_urban.xlsx"
PATH_URB_POP      = "/mnt/data/unhabitat_urban_population_midyear.xlsx"
PATH_LAND_CONS    = "/mnt/data/unhabitat_land_consumption_rate.xlsx"

# ---------------------------------------------
# 2) Carregar dataset principal
# ---------------------------------------------

df_main = pd.read_csv(PATH_MAIN)

print("Shape df_main (antes do UN-Habitat):", df_main.shape)
print(df_main.columns.tolist()[:20])

# Vamos assumir que seu df_main tem algo como:
# - 'Country Name' (nome do país em texto)
# - 'Year'         (ano numérico)
# Ajuste aqui se seus nomes forem outros:
COUNTRY_COL_MAIN = "Country Name"
YEAR_COL_MAIN    = "Year"

# ---------------------------------------------
# 3) Funções auxiliares para padronizar país/ano
# ---------------------------------------------

def normalize_country_name(s):
    """Normaliza nomes de países para facilitar o merge por texto."""
    if pd.isna(s):
        return np.nan
    s = str(s).strip().upper()
    # remove caracteres muito especiais
    for ch in [".", ",", ";", ":", "'", '"', "´", "`"]:
        s = s.replace(ch, "")
    s = " ".join(s.split())  # remove espaços duplicados
    return s

def prepare_unhabitat_generic(
    path,
    country_cols=("Country or Area", "Country Name", "COUNTRY", "Country"),
    year_cols=("Year", "Time", "TIME_PERIOD"),
    value_cols=("Value", "OBS_VALUE", "Indicator Value"),
    indicator_name="indicator"
):
    """
    Lê um arquivo UN-Habitat (Excel/CSV) e retorna um df padronizado:
    ['country_key', 'Year', <indicator_name>]
    Tentamos algumas opções de nomes de colunas típicas.
    """
    # Detectar se é Excel ou CSV
    if path.lower().endswith((".xlsx", ".xls")):
        df_raw = pd.read_excel(path)
    else:
        df_raw = pd.read_csv(path)
    
    df = df_raw.copy()

    # 1) Detectar coluna de país
    country_col = None
    for c in country_cols:
        if c in df.columns:
            country_col = c
            break
    if country_col is None:
        raise ValueError(f"Não foi encontrada coluna de país em {path}. Colunas disponíveis: {df.columns.tolist()}")

    # 2) Detectar coluna de ano
    year_col = None
    for c in year_cols:
        if c in df.columns:
            year_col = c
            break
    if year_col is None:
        raise ValueError(f"Não foi encontrada coluna de ano em {path}. Colunas disponíveis: {df.columns.tolist()}")

    # 3) Detectar coluna de valor
    value_col = None
    for c in value_cols:
        if c in df.columns:
            value_col = c
            break
    if value_col is None:
        # Se tiver exatamente uma coluna numérica principal, você pode
        # tentar identificá-la automaticamente:
        num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if len(num_cols) == 1:
            value_col = num_cols[0]
        else:
            raise ValueError(f"Não foi encontrada coluna de valor em {path}. Colunas disponíveis: {df.columns.tolist()}")

    # 4) Selecionar e renomear
    df = df[[country_col, year_col, value_col]].copy()
    df.columns = ["Country_UNH", "Year", indicator_name]

    # 5) Limpeza básica
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
    df = df.dropna(subset=["Year"])

    # 6) Criar chave de país normalizada
    df["country_key"] = df["Country_UNH"].apply(normalize_country_name)

    # Remover duplicados (se houver vários registros por país-ano, pegamos a média)
    df = (
        df.groupby(["country_key", "Year"], as_index=False)[indicator_name]
        .mean()
    )

    return df

# ---------------------------------------------
# 4) Preparar cada indicador UN-Habitat
#    (ajuste os 'indicator_name' se quiser renomear)
# ---------------------------------------------

df_slums = prepare_unhabitat_generic(
    PATH_SLUMS,
    indicator_name="uh_slums_pct"
)

df_water = prepare_unhabitat_generic(
    PATH_WATER,
    indicator_name="uh_water_access_pct"
)

df_san = prepare_unhabitat_generic(
    PATH_SANITATION,
    indicator_name="uh_sanitation_access_pct"
)

df_green = prepare_unhabitat_generic(
    PATH_GREEN,
    indicator_name="uh_green_area_m2_per_capita"
)

df_pm25 = prepare_unhabitat_generic(
    PATH_PM25,
    indicator_name="uh_pm25_ug_m3"
)

df_urbpop = prepare_unhabitat_generic(
    PATH_URB_POP,
    indicator_name="uh_urban_population_thousands"
)

df_land = prepare_unhabitat_generic(
    PATH_LAND_CONS,
    indicator_name="uh_land_consumption_rate"
)

print("UN-Habitat: shapes individuais")
print("Slums       :", df_slums.shape)
print("Water       :", df_water.shape)
print("Sanitation  :", df_san.shape)
print("Green Area  :", df_green.shape)
print("PM2.5       :", df_pm25.shape)
print("Urban Pop   :", df_urbpop.shape)
print("Land Cons   :", df_land.shape)

# ---------------------------------------------
# 5) Criar chave de país no df_main
# ---------------------------------------------

df_main["country_key"] = df_main[COUNTRY_COL_MAIN].apply(normalize_country_name)

# Se o ano no df_main não estiver em uma coluna chamada "Year", ajuste YEAR_COL_MAIN acima
if YEAR_COL_MAIN != "Year":
    df_main = df_main.rename(columns={YEAR_COL_MAIN: "Year"})

# ---------------------------------------------
# 6) Fazer merge incremental com todos os indicadores
# ---------------------------------------------

df_enriched = df_main.copy()

for df_ind, ind_name in [
    (df_slums, "uh_slums_pct"),
    (df_water, "uh_water_access_pct"),
    (df_san, "uh_sanitation_access_pct"),
    (df_green, "uh_green_area_m2_per_capita"),
    (df_pm25, "uh_pm25_ug_m3"),
    (df_urbpop, "uh_urban_population_thousands"),
    (df_land, "uh_land_consumption_rate"),
]:
    print(f"Merging indicador: {ind_name}")
    df_enriched = df_enriched.merge(
        df_ind[["country_key", "Year", ind_name]],
        on=["country_key", "Year"],
        how="left"
    )

print("Shape df_enriched (após UN-Habitat):", df_enriched.shape)

# ---------------------------------------------
# 7) Verificar cobertura dos novos indicadores
# ---------------------------------------------

novas_cols = [
    "uh_slums_pct",
    "uh_water_access_pct",
    "uh_sanitation_access_pct",
    "uh_green_area_m2_per_capita",
    "uh_pm25_ug_m3",
    "uh_urban_population_thousands",
    "uh_land_consumption_rate",
]

print(df_enriched[novas_cols].describe(include="all"))

# Percentual de missing por novo indicador
missing_summary = df_enriched[novas_cols].isna().mean().sort_values()
print("\nPercentual de missing por indicador UN-Habitat:")
print((missing_summary * 100).round(1).astype(str) + "%")


# ---------------------------------------------
# 8) Salvar resultado enriquecido (para uso nos próximos capítulos)
# ---------------------------------------------

OUTPUT_PATH = "/mnt/data/wdi_merged_unhabitat.csv"
df_enriched.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")

print(f"\nArquivo salvo em: {OUTPUT_PATH}")